## V6: Параметрический перебор признаков (feature sweeper)

Задача: быстро проверять разные наборы признаков по всем установкам (`installation`),
обучая отдельные модели на каждую установку и сохраняя метрики вместе с конфигурацией признаков.

Как использовать:
1) В блоке "Конфигурация" указать список экспериментов `sweeps` c именем и списком ключей признаков.
2) Запустить ячейки по порядку; результаты сохранятся в `data/param_sweeps/`.


In [56]:
# Импорты и данные
import os, sys, json, time
import numpy as np
import pandas as pd

if '../src' not in sys.path:
    sys.path.append('../src')
from database import load_corrosion_data

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

print('Библиотеки загружены')

Библиотеки загружены


In [5]:
print('Загружаю данные...')
df = load_corrosion_data()
print(f"Всего записей: {len(df):,}")

Загружаю данные...
Всего записей: 442,052


In [57]:
TARGET = 'corrosion_rate'
assert TARGET in df.columns

# Базовая фильтрация цели
before = len(df)
df = df[df[TARGET].notna()]
df = df[(df[TARGET] >= 0) & (df[TARGET] <= 10)]
print(f"Готово. Осталось: {len(df):,} из {before:,}")

key_col = 'installation'
print('Группировка по:', key_col)


Готово. Осталось: 97,481 из 97,481
Группировка по: installation


In [58]:
# Фильтр по списку установок (опционально)
# Укажите нужные установки. Если список пустой, используется весь корпус.
selected_installations = ['АВТ-5','KK-2']  # например: ['АВТ-1', 'АВТ-5']

if key_col == 'installation' and len(selected_installations) > 0:
    df = df[df['installation'].astype(str).isin(selected_installations)].copy()
    print(f"Применён фильтр по installations, осталось: {len(df):,}")
else:
    print('Фильтр по installations не применён')

Применён фильтр по installations, осталось: 97,481


In [71]:
# Конфигурация: базовые и опциональные признаки
#
# Справочник по признакам из SQL-представления (см. pipeline_corrosion_analysis_view.sql):
#
# ГЕОМЕТРИЧЕСКИЕ (geometry):
# - wall_thickness                    -> толщина стенки (outer_diameter - inner_diameter)
# - radius                            -> outer_diameter / 2.0
# - outer_diameter, inner_diameter    -> диаметры
# - diameter_to_thickness_ratio       -> отношение диаметра к толщине стенки
# - cross_sectional_area              -> площадь поперечного сечения
#
# МАТЕРИАЛ (materials):
# - material_type                     -> тип материала (категория)
# - material_resistance_score         -> числовая оценка стойкости материала (CASE по типу стали)
# - material_code                     -> код материала
#
# ОПЕРАЦИОННЫЕ (operational):
# - operating_temperature             -> рабочая температура
# - operating_pressure                -> рабочее давление
# - start_date_of_operation           -> начало эксплуатации (для возраста)
# - measurement_date                  -> дата измерения (для возраста)
#
# ТИП КОМПОНЕНТА (component type):
# - component_type_id                 -> id типа
# - component_type_name               -> имя типа (категория)
#
# ХИМИЯ — ОСНОВНЫЕ (chem_basic):
# - water_content, h2s_content, sulfur_content, chlorine_content,
#   co2_content, oxygen_content, nitrogen_content, hydrogen_content
#
# ХИМИЯ — УГЛЕВОДОРОДЫ (chem_hydrocarbons):
# - methane_content, ethane_content, propane_content, butane_content, isobutane_content,
#   pentane_content, isopentane_content, gasoline_c6_c8_content, hexane_content,
#   heavy_naphtha_content, kerosene_content, diesel_content, residues_content,
#   propylene_content, ethylene_content, butylene_content
#
# ХИМИЯ — КИСЛОТЫ И ПРОЧЕЕ (chem_acids_others):
# - sulfuric_acid_content, hydrochloric_acid_content, acetic_acid_content, naphthenic_acid_content,
#   ammonia_content, ammonium_content, hydrogen_fluoride_content,
#   sodium_hydroxide_content, corrosion_inhibitor_content
#
# АГРЕГАТЫ (aggregates):
# - total_components, total_composition, total_sulfur_compounds,
#   total_chlorine_compounds, total_acids
#
# МЕТА (идентификаторы/измерения):
# - id, installation, equipment, mms, contour, component,
#   nominal_thickness_mmc, tmin_mmc
#
# КЛЮЧИ ДЛЯ СВИПЕРА (mapping ключ -> колонка):
# - 'area'            -> cross_sectional_area
# - 'material'        -> material_resistance_score
# - 'temp'            -> operating_temperature
# - 'press'           -> operating_pressure
# - 'chem_sulfur'     -> sulfur_content
# - 'chem_total_s'    -> total_sulfur_compounds
# - 'chem_h2s'        -> h2s_content
# - 'chem_water'      -> water_content
# - 'chem_oxygen'     -> oxygen_content
# - 'int_temp_water'  -> water_content × operating_temperature
# - 'int_temp_oxygen' -> oxygen_content × operating_temperature

base_keys = [
    'area',        # cross_sectional_area
    'material',    # material_resistance_score
    'temp',        # operating_temperature
    'press'        # operating_pressure
]

optional_keys = [
    # Базовые химические
    'chem_sulfur', 'chem_total_s', 'chem_h2s', 'chem_water', 'chem_oxygen', 'chem_chlorine', 'chem_co2',
    'chem_nitrogen', 'chem_hydrogen',
    # Углеводороды
    'chem_methane','chem_ethane','chem_propane','chem_butane','chem_isobutane',
    'chem_pentane','chem_isopentane','chem_gasoline_c6_c8','chem_hexane',
    'chem_heavy_naphtha','chem_kerosene','chem_diesel','chem_residues',
    'chem_propylene','chem_ethylene','chem_butylene',
    # Кислоты/прочее
    'chem_sulfuric_acid','chem_hydrochloric_acid','chem_acetic_acid','chem_naphthenic_acid',
    'chem_ammonia','chem_ammonium','chem_hydrogen_fluoride','chem_sodium_hydroxide','chem_corrosion_inhibitor',
    # Агрегаты (химические)
    'chem_total_chlorine','chem_total_acids',
    # Интеракции
    'int_temp_water','int_temp_oxygen'
]

# Список всех химических ключей (без интеракций)
all_chem_keys = [k for k in optional_keys if k.startswith('chem_')]

# Примеры экспериментов
def_sweeps = [
    {'name': 'baseline_min', 'keys': ['area','material','temp','press']},
    {'name': 'add_sulfur',   'keys': ['area','material','temp','press','chem_sulfur','chem_total_s']},
    {'name': 'add_h2s',      'keys': ['area','material','temp','press','chem_h2s']},
    {'name': 'add_water_O2', 'keys': ['area','material','temp','press','chem_water','chem_oxygen']},
        {'name': '6', 'keys': ['area','material','temp','press','chem_water','chem_oxygen','chem_sulfur','chem_total_s','chem_h2s']},

]

# Пользователь может редактировать список ниже перед запуском
sweeps = def_sweeps


In [72]:
# Генерация X,y по ключам

def build_xy_by_keys(df_in: pd.DataFrame, keys: list) -> (pd.DataFrame, pd.Series):
    cols = []
    # База
    if 'area' in keys and 'cross_sectional_area' in df_in.columns:
        cols.append('cross_sectional_area')
    if 'material' in keys and 'material_resistance_score' in df_in.columns:
        cols.append('material_resistance_score')
    if 'temp' in keys and 'operating_temperature' in df_in.columns:
        cols.append('operating_temperature')
    if 'press' in keys and 'operating_pressure' in df_in.columns:
        cols.append('operating_pressure')

    # Химия — расширенный мэппинг
    mapping = {
        'chem_sulfur': 'sulfur_content',
        'chem_total_s': 'total_sulfur_compounds',
        'chem_h2s': 'h2s_content',
        'chem_water': 'water_content',
        'chem_oxygen': 'oxygen_content',
        'chem_chlorine': 'chlorine_content',
        'chem_co2': 'co2_content',
        'chem_nitrogen': 'nitrogen_content',
        'chem_hydrogen': 'hydrogen_content',
        # Углеводороды
        'chem_methane': 'methane_content',
        'chem_ethane': 'ethane_content',
        'chem_propane': 'propane_content',
        'chem_butane': 'butane_content',
        'chem_isobutane': 'isobutane_content',
        'chem_pentane': 'pentane_content',
        'chem_isopentane': 'isopentane_content',
        'chem_gasoline_c6_c8': 'gasoline_c6_c8_content',
        'chem_hexane': 'hexane_content',
        'chem_heavy_naphtha': 'heavy_naphtha_content',
        'chem_kerosene': 'kerosene_content',
        'chem_diesel': 'diesel_content',
        'chem_residues': 'residues_content',
        'chem_propylene': 'propylene_content',
        'chem_ethylene': 'ethylene_content',
        'chem_butylene': 'butylene_content',
        # Кислоты и прочее
        'chem_sulfuric_acid': 'sulfuric_acid_content',
        'chem_hydrochloric_acid': 'hydrochloric_acid_content',
        'chem_acetic_acid': 'acetic_acid_content',
        'chem_naphthenic_acid': 'naphthenic_acid_content',
        'chem_ammonia': 'ammonia_content',
        'chem_ammonium': 'ammonium_content',
        'chem_hydrogen_fluoride': 'hydrogen_fluoride_content',
        'chem_sodium_hydroxide': 'sodium_hydroxide_content',
        'chem_corrosion_inhibitor': 'corrosion_inhibitor_content',
        # Агрегаты (относящиеся к химии)
        'chem_total_chlorine': 'total_chlorine_compounds',
        'chem_total_acids': 'total_acids',
    }
    for k, c in mapping.items():
        if k in keys and c in df_in.columns:
            cols.append(c)

    X = df_in[cols].copy()
    # Импутация: химия -> 0, прочие -> медиана
    chem_set = set(mapping.values())
    for c in X.columns:
        if c in chem_set:
            X[c] = X[c].fillna(0)
        else:
            X[c] = X[c].fillna(X[c].median())

    # Интеракции
    if 'int_temp_water' in keys and {'operating_temperature','water_content'}.issubset(X.columns):
        X['water_content_x_operating_temperature'] = X['water_content'] * X['operating_temperature']
    if 'int_temp_oxygen' in keys and {'operating_temperature','oxygen_content'}.issubset(X.columns):
        X['oxygen_content_x_operating_temperature'] = X['oxygen_content'] * X['operating_temperature']

    y = df_in[TARGET].astype(float)
    return X, y


In [73]:
# Оценка по всем installations для одного конфига

def eval_per_installation(df_in: pd.DataFrame, keys: list, min_samples: int = 150):
    if key_col is None:
        raise ValueError('Нет поля для группировки (installation/equipment)')

    res_rows = []
    unique_keys = df_in[key_col].dropna().astype(str).unique()

    X_full, y_full = build_xy_by_keys(df_in, keys)

    for k in sorted(unique_keys):
        mask = df_in[key_col].astype(str) == k
        n = int(mask.sum())
        if n < min_samples:
            continue
        Xk = X_full.loc[mask]
        yk = y_full.loc[mask]

        X_tr, X_va, y_tr, y_va = train_test_split(Xk, yk, test_size=0.2, random_state=42)

        ridge = Pipeline([('scaler', StandardScaler(with_mean=False)), ('model', Ridge(alpha=1.0, random_state=42))])
        ridge.fit(X_tr, y_tr)
        pr = ridge.predict(X_va)

        rf = RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)
        rf.fit(X_tr, y_tr)
        prf = rf.predict(X_va)

        res_rows.append({
            key_col: k,
            'n_samples': n,
            'Ridge_MAE': float(mean_absolute_error(y_va, pr)),
            'Ridge_RMSE': float(np.sqrt(mean_squared_error(y_va, pr))),
            'Ridge_R2': float(r2_score(y_va, pr)),
            'RF_MAE': float(mean_absolute_error(y_va, prf)),
            'RF_RMSE': float(np.sqrt(mean_squared_error(y_va, prf))),
            'RF_R2': float(r2_score(y_va, prf))
        })

    return pd.DataFrame(res_rows)


In [74]:
# Запуск перебора и сохранение результатов

out_dir = '../data/param_sweeps'
os.makedirs(out_dir, exist_ok=True)

all_runs = []
for cfg in sweeps:
    name = cfg['name']
    keys = cfg['keys']
    print(f'Эксперимент: {name} -> {keys}')
    t0 = time.time()
    res_df = eval_per_installation(df, keys)
    dt = time.time() - t0

    # агрегаты по установкам
    agg = res_df[['Ridge_R2','RF_R2','Ridge_MAE','RF_MAE']].mean().to_dict()
    agg['n_installations'] = int(len(res_df))
    agg['elapsed_sec'] = round(dt, 2)

    # сохранение
    stamp = int(time.time())
    base = f'{stamp}_{name}'
    res_path_csv = os.path.join(out_dir, base + '.csv')
    res_path_json = os.path.join(out_dir, base + '.json')
    res_df.to_csv(res_path_csv, index=False, encoding='utf-8')

    payload = {
        'name': name,
        'keys': keys,
        'agg': agg,
        'per_installation_csv': res_path_csv
    }
    with open(res_path_json, 'w', encoding='utf-8') as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

    all_runs.append(payload)
    print('Готово:', payload)

# Итоговый сводный файл
summary_path = os.path.join(out_dir, 'summary_runs.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(all_runs, f, ensure_ascii=False, indent=2)
print('Сводка сохранена в', summary_path)


Эксперимент: baseline_min -> ['area', 'material', 'temp', 'press']
Готово: {'name': 'baseline_min', 'keys': ['area', 'material', 'temp', 'press'], 'agg': {'Ridge_R2': 0.024135425009265354, 'RF_R2': 0.27428429612080296, 'Ridge_MAE': 0.04687938408085368, 'RF_MAE': 0.042688979964645074, 'n_installations': 2, 'elapsed_sec': 2.23}, 'per_installation_csv': '../data/param_sweeps\\1761586830_baseline_min.csv'}
Эксперимент: add_sulfur -> ['area', 'material', 'temp', 'press', 'chem_sulfur', 'chem_total_s']
Готово: {'name': 'add_sulfur', 'keys': ['area', 'material', 'temp', 'press', 'chem_sulfur', 'chem_total_s'], 'agg': {'Ridge_R2': 0.025903352722483064, 'RF_R2': 0.2778217220531271, 'Ridge_MAE': 0.04690201647726562, 'RF_MAE': 0.04261323664877472, 'n_installations': 2, 'elapsed_sec': 2.76}, 'per_installation_csv': '../data/param_sweeps\\1761586833_add_sulfur.csv'}
Эксперимент: add_h2s -> ['area', 'material', 'temp', 'press', 'chem_h2s']
Готово: {'name': 'add_h2s', 'keys': ['area', 'material', 'te

In [75]:
# Постобработка: единая таблица и сводные таблицы (без отбора лучших)

# Собираем все результаты из JSON-файлов текущего запуска
import glob
from pathlib import Path

out_dir = '../data/param_sweeps'
# Берём только файлы экспериментов вида "{stamp}_{name}.json", исключая сводный файл summary_runs.json
json_paths = [p for p in sorted(glob.glob(os.path.join(out_dir, '*_*.json'))) if Path(p).name != 'summary_runs.json']

combined_rows = []
for jp in json_paths:
    with open(jp, 'r', encoding='utf-8') as f:
        payload = json.load(f)
    # Пропускаем, если структура не соответствует ожидаемой (например, список)
    if not isinstance(payload, dict):
        continue
    name = payload.get('name')
    keys = payload.get('keys', [])
    csv_path = payload.get('per_installation_csv')
    if not csv_path or not os.path.exists(csv_path):
        continue
    df_part = pd.read_csv(csv_path)
    df_part['experiment'] = name
    df_part['keys_str'] = ','.join(keys)
    combined_rows.append(df_part)

if combined_rows:
    combined = pd.concat(combined_rows, ignore_index=True)
else:
    combined = pd.DataFrame()

print('Единая таблица результатов:', combined.shape)

# Сохраняем объединенную таблицу (все строки без агрегации и отбора)
all_csv = os.path.join(out_dir, 'all_results.csv')
combined.to_csv(all_csv, index=False, encoding='utf-8')
print('Сохранено:', all_csv)

# Пивоты по R2: строки — установки, столбцы — эксперименты
if not combined.empty:
    key_col_here = 'installation' if 'installation' in combined.columns else ('equipment' if 'equipment' in combined.columns else None)
    if key_col_here is None:
        raise ValueError('Нет колонки для группировки (installation/equipment) в объединенной таблице')

    pivot_rf = combined.pivot_table(index=key_col_here, columns='experiment', values='RF_R2', aggfunc='mean')
    pivot_ridge = combined.pivot_table(index=key_col_here, columns='experiment', values='Ridge_R2', aggfunc='mean')
    pivot_rf_csv = os.path.join(out_dir, 'pivot_rf_r2.csv')
    pivot_ridge_csv = os.path.join(out_dir, 'pivot_ridge_r2.csv')
    pivot_rf.to_csv(pivot_rf_csv, encoding='utf-8')
    pivot_ridge.to_csv(pivot_ridge_csv, encoding='utf-8')
    print('Сохранены пивоты:', pivot_rf_csv, 'и', pivot_ridge_csv)

    # Отображение: показываем первые строки объединенной таблицы и пивотов
    display(combined.head(100))
    display(pivot_rf.head(100))
    display(pivot_ridge.head(100))
else:
    print('Нет результатов для объединения — выполните предыдущую ячейку перебора.')

Единая таблица результатов: (10, 10)
Сохранено: ../data/param_sweeps\all_results.csv
Сохранены пивоты: ../data/param_sweeps\pivot_rf_r2.csv и ../data/param_sweeps\pivot_ridge_r2.csv


,installation,n_samples,Ridge_MAE,Ridge_RMSE,Ridge_R2,RF_MAE,RF_RMSE,RF_R2,experiment,keys_str
0,KK-2,37814,0.047483,0.077591,0.013096,0.043637,0.064740,0.312941,baseline_min,"area,material,temp,press"
1,АВТ-5,59667,0.046276,0.090378,0.035175,0.041741,0.080444,0.235628,baseline_min,"area,material,temp,press"
2,KK-2,37814,0.047472,0.077460,0.016433,0.043551,0.064479,0.318468,add_sulfur,"area,material,temp,press,chem_sulfur,chem_total_s"
3,АВТ-5,59667,0.046332,0.090369,0.035374,0.041675,0.080362,0.237175,add_sulfur,"area,material,temp,press,chem_sulfur,chem_total_s"
4,KK-2,37814,0.047444,0.077483,0.015831,0.043538,0.064472,0.318623,add_h2s,"area,material,temp,press,chem_h2s"
5,АВТ-5,59667,0.046277,0.090379,0.035171,0.041672,0.080363,0.237161,add_h2s,"area,material,temp,press,chem_h2s"
6,KK-2,37814,0.047487,0.077580,0.013382,0.043519,0.064454,0.319001,add_water_O2,"area,material,temp,press,chem_water,chem_oxygen"
7,АВТ-5,59667,0.046288,0.090373,0.035288,0.041700,0.080376,0.236914,add_water_O2,"area,material,temp,press,chem_water,chem_oxygen"
8,KK-2,37814,0.047456,0.077336,0.019585,0.043524,0.064471,0.318639,6,"area,material,temp,press,chem_water,chem_oxyge..."
9,АВТ-5,59667,0.046348,0.090325,0.036306,0.041684,0.080367,0.237094,6,"area,material,temp,press,chem_water,chem_oxyge..."


experiment,6,add_h2s,add_sulfur,add_water_O2,baseline_min
installation,,,,,
KK-2,0.318639,0.318623,0.318468,0.319001,0.312941
АВТ-5,0.237094,0.237161,0.237175,0.236914,0.235628


experiment,6,add_h2s,add_sulfur,add_water_O2,baseline_min
installation,,,,,
KK-2,0.019585,0.015831,0.016433,0.013382,0.013096
АВТ-5,0.036306,0.035171,0.035374,0.035288,0.035175
